# Option 1: Fix EfficientNet with Proper Hyperparameters

## Problem
EfficientNet-B0 underperformed (Val R²=0.5903) because it used ResNet18's hyperparameters.

**Key Issue**: EfficientNet has 1280 features vs ResNet18's 512 (2.5x more!)

## Solution
Test 3 EfficientNet variants with higher learning rates:

1. **HigherLR**: head_lr=1e-3 (3.3x), backbone_lr=5e-5 (5x), less regularization
2. **VeryHighLR**: head_lr=2e-3 (6.7x), backbone_lr=1e-4 (10x) - aggressive!
3. **WithBatchNorm**: Same as HigherLR but adds BatchNorm to head

## Expected
Best case: Val R²=0.64-0.68 (could beat baseline 0.6852!)

## Time
- DEBUG: ~15 min (2 epochs × 3 variants)
- FULL: ~1.5 hours (40 epochs × 3 variants)

---

## Part 1: Setup & Data Preparation

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm.auto import tqdm
import copy
from datetime import datetime

# Install timm
try:
    import timm
    TIMM_AVAILABLE = True
    print("✅ timm library available")
except ImportError:
    print("⏳ Installing timm...")
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "timm", "-q"])
    import timm
    TIMM_AVAILABLE = True
    print("✅ timm installed")

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ Setup complete")

✅ timm library available
Device: cpu
✓ Setup complete


In [24]:
# ================================================================
# CONFIGURATION: 3 EFFICIENTNET TUNING EXPERIMENTS
# ================================================================

# 🐛 DEBUG MODE
DEBUG_MODE = False  # Set to False for full training

BATCH_SIZE = 16
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
COMPETITION_WEIGHTS = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5])

if DEBUG_MODE:
    print("\n" + "🐛"*40)
    print("DEBUG MODE: 2 epochs/phase for testing")
    print("🐛"*40 + "\n")

def get_epochs(full):
    return 2 if DEBUG_MODE else full

# Three EfficientNet variants
VARIATIONS = {
    'EfficientNet_HigherLR': {
        'phase1_epochs': get_epochs(15),
        'phase2_epochs': get_epochs(40),
        'phase1_lr': 3e-4,
        'phase2_lr': 1e-3,  # 3.3x higher
        'phase2_backbone_lr': 5e-5,  # 5x higher
        'weight_decay': 5e-5,  # Less regularization
        'hidden_dim': 256,
        'dropout': 0.15,  # Less dropout
        'backbone': 'efficientnet_b0',
        'use_scheduler': False,
        'use_full_norm': False,
        'use_batch_norm': False,
        'requires_timm': True,
        'description': 'EfficientNet with 3.3x higher LR',
        'expected_kaggle': '0.52-0.54'
    },
    'EfficientNet_VeryHighLR': {
        'phase1_epochs': get_epochs(15),
        'phase2_epochs': get_epochs(40),
        'phase1_lr': 3e-4,
        'phase2_lr': 2e-3,  # 6.7x higher!
        'phase2_backbone_lr': 1e-4,  # 10x higher!
        'weight_decay': 5e-5,
        'hidden_dim': 256,
        'dropout': 0.15,
        'backbone': 'efficientnet_b0',
        'use_scheduler': False,
        'use_full_norm': False,
        'use_batch_norm': False,
        'requires_timm': True,
        'description': 'EfficientNet with aggressive 6.7x LR',
        'expected_kaggle': '0.51-0.53'
    },
    'EfficientNet_WithBatchNorm': {
        'phase1_epochs': get_epochs(15),
        'phase2_epochs': get_epochs(40),
        'phase1_lr': 3e-4,
        'phase2_lr': 1e-3,
        'phase2_backbone_lr': 5e-5,
        'weight_decay': 5e-5,
        'hidden_dim': 256,
        'dropout': 0.15,
        'backbone': 'efficientnet_b0',
        'use_scheduler': False,
        'use_full_norm': False,
        'use_batch_norm': True,  # Added BatchNorm
        'requires_timm': True,
        'description': 'EfficientNet with BatchNorm in head',
        'expected_kaggle': '0.52-0.54'
    }
}

print("="*80)
print("EFFICIENTNET TUNING EXPERIMENTS")
print("="*80)
print(f"\nExperiments ({len(VARIATIONS)}):")
for name, cfg in VARIATIONS.items():
    ep = f"P1:{cfg['phase1_epochs']}ep P2:{cfg['phase2_epochs']}ep"
    lr = f"head_lr={cfg['phase2_lr']:.0e} backbone_lr={cfg['phase2_backbone_lr']:.0e}"
    print(f"  {name}: {ep}")
    print(f"     {lr}, Expected Kaggle: {cfg['expected_kaggle']}")
print(f"\n⏱️ {'DEBUG: ~15 min' if DEBUG_MODE else 'FULL: ~1.5 hours'}")
print("="*80)

EFFICIENTNET TUNING EXPERIMENTS

Experiments (3):
  EfficientNet_HigherLR: P1:15ep P2:40ep
     head_lr=1e-03 backbone_lr=5e-05, Expected Kaggle: 0.52-0.54
  EfficientNet_VeryHighLR: P1:15ep P2:40ep
     head_lr=2e-03 backbone_lr=1e-04, Expected Kaggle: 0.51-0.53
  EfficientNet_WithBatchNorm: P1:15ep P2:40ep
     head_lr=1e-03 backbone_lr=5e-05, Expected Kaggle: 0.52-0.54

⏱️ FULL: ~1.5 hours


In [25]:
# Load data
train_enriched = pd.read_csv('../../competition/train_enriched.csv')
train_enriched['Sampling_Date'] = pd.to_datetime(train_enriched['Sampling_Date'])
train_enriched['full_image_path'] = train_enriched['image_path'].apply(lambda x: f'../../competition/{x}')

print(f"Full dataset: {len(train_enriched)} images")

# Split
train_data, val_data = train_test_split(train_enriched, test_size=0.2, random_state=42)

# Calculate normalization stats (split-based, since use_full_norm=False for all)
target_means = torch.tensor([train_data[col].mean() for col in TARGET_COLS], dtype=torch.float32)
target_stds = torch.tensor([train_data[col].std() for col in TARGET_COLS], dtype=torch.float32)

print("\nNormalization (train split, 285 images):")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: mean={target_means[i]:.2f}g, std={target_stds[i]:.2f}g")

print(f"\n✓ Split: {len(train_data)} train, {len(val_data)} val")

Full dataset: 357 images

Normalization (train split, 285 images):
  Dry_Green_g    : mean=27.49g, std=26.19g
  Dry_Dead_g     : mean=12.01g, std=12.50g
  Dry_Clover_g   : mean=6.26g, std=11.75g
  GDM_g          : mean=33.74g, std=25.62g
  Dry_Total_g    : mean=45.75g, std=28.86g

✓ Split: 285 train, 72 val


In [26]:
# Prepare tabular features
weather_features = [
    'rainfall_7d', 'rainfall_30d',
    'temp_max_7d', 'temp_min_7d', 'temp_mean_7d', 'temp_mean_30d', 'temp_range_7d',
    'et0_7d', 'et0_30d',
    'water_balance_7d', 'water_balance_30d',
    'days_since_rain', 'daylength', 'season'
]

continuous_features = weather_features + ['Pre_GSHH_NDVI', 'Height_Ave_cm']
scaler = StandardScaler()
train_data[continuous_features] = scaler.fit_transform(train_data[continuous_features])
val_data[continuous_features] = scaler.transform(val_data[continuous_features])

le_state = LabelEncoder()
le_species = LabelEncoder()
train_data['State_encoded'] = le_state.fit_transform(train_data['State'])
train_data['Species_encoded'] = le_species.fit_transform(train_data['Species'])
val_data['State_encoded'] = le_state.transform(val_data['State'])
val_data['Species_encoded'] = le_species.transform(val_data['Species'])

num_states = len(le_state.classes_)
num_species = len(le_species.classes_)

print(f"✓ Tabular features prepared")
print(f"  States: {num_states}, Species: {num_species}")

✓ Tabular features prepared
  States: 4, Species: 15


### Dataset Classes

In [27]:
class NormalizedDataset(Dataset):
    """Image-only dataset with normalized targets."""
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds
        
        transform_list = [transforms.Resize((224, 224))]
        
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        self.transform = transforms.Compose(transform_list)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        
        return {
            'image': img,
            'targets': targets_normalized,
            'targets_original': targets
        }

class MultimodalDataset(Dataset):
    """Multimodal dataset for auxiliary pretraining."""
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds
        
        transform_list = [transforms.Resize((224, 224))]
        
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        self.transform = transforms.Compose(transform_list)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Image
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        
        # Tabular features
        weather = torch.tensor(row[weather_features].values.astype('float32'), dtype=torch.float32)
        ndvi = torch.tensor(row['Pre_GSHH_NDVI'], dtype=torch.float32)
        height = torch.tensor(row['Height_Ave_cm'], dtype=torch.float32)
        state = torch.tensor(row['State_encoded'], dtype=torch.long)
        species = torch.tensor(row['Species_encoded'], dtype=torch.long)
        
        # Targets
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        
        return {
            'image': img,
            'weather': weather,
            'ndvi': ndvi,
            'height': height,
            'state': state,
            'species': species,
            'targets': targets_normalized,
            'targets_original': targets
        }

print("✓ Dataset classes defined")
print("✓ Datasets will be created for each experiment with appropriate normalization")

✓ Dataset classes defined
✓ Datasets will be created for each experiment with appropriate normalization


### Model Architecture

In [28]:
def get_backbone_and_features(backbone_name):
    """FIXED timm support."""
    if backbone_name == 'resnet18':
        model = models.resnet18(pretrained=True)
        backbone = nn.Sequential(*list(model.children())[:-1])
        feature_dim = 512
    elif backbone_name == 'densenet121':
        model = models.densenet121(pretrained=True)
        backbone = nn.Sequential(model.features, nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1)))
        feature_dim = 1024
    elif backbone_name in ['efficientnet_b0', 'mobilenetv3_large_100']:
        if not TIMM_AVAILABLE:
            raise ImportError(f"timm required for {backbone_name}")
        import timm

        class TimmBackbone(nn.Module):
            def __init__(self, model):
                super().__init__()
                self.model = model
            def forward(self, x):
                features = self.model.forward_features(x)
                if len(features.shape) == 4:
                    features = features.mean([2, 3])
                return features.unsqueeze(-1).unsqueeze(-1)

        model = timm.create_model(backbone_name, pretrained=True)
        backbone = TimmBackbone(model)
        feature_dim = 1280 if backbone_name == 'efficientnet_b0' else 960
    else:
        raise ValueError(f"Unknown: {backbone_name}")
    return backbone, feature_dim

class AuxiliaryPretrainedModel(nn.Module):
    def __init__(self, backbone_name='resnet18', num_outputs=5, hidden_dim=256,
                 dropout=0.2, num_states=4, num_species=15, use_batch_norm=False):
        super().__init__()
        self.backbone, feature_dim = get_backbone_and_features(backbone_name)
        self.backbone_name = backbone_name
        self.feature_dim = feature_dim
        
        # Auxiliary heads
        self.ndvi_head = nn.Linear(feature_dim, 1)
        self.height_head = nn.Linear(feature_dim, 1)
        self.weather_head = nn.Linear(feature_dim, 14)
        self.state_head = nn.Linear(feature_dim, num_states)
        self.species_head = nn.Linear(feature_dim, num_species)
        
        # Biomass head with optional BatchNorm
        if use_batch_norm:
            self.biomass_head = nn.Sequential(
                nn.Linear(feature_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, num_outputs)
            )
        else:
            self.biomass_head = nn.Sequential(
                nn.Linear(feature_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, num_outputs)
            )
    
    def forward(self, x, mode='biomass'):
        features = self.backbone(x).flatten(1)
        if mode == 'auxiliary':
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'weather': self.weather_head(features),
                'state': self.state_head(features),
                'species': self.species_head(features)
            }
        else:
            return self.biomass_head(features)

print("✓ Model with BatchNorm support")

✓ Model with BatchNorm support


### Utility Functions

In [29]:
def calculate_competition_r2(predictions, targets):
    """Calculate competition R² (weighted)."""
    per_target_r2 = []
    competition_r2 = 0
    
    weights = COMPETITION_WEIGHTS.numpy()
    
    for i in range(5):
        r2 = r2_score(targets[:, i], predictions[:, i])
        per_target_r2.append(r2)
        competition_r2 += weights[i] * r2
    
    return competition_r2, per_target_r2

def auxiliary_loss(predictions, targets):
    """Multi-task loss for predicting tabular features."""
    loss_ndvi = F.mse_loss(predictions['ndvi'].squeeze(), targets['ndvi'])
    loss_height = F.mse_loss(predictions['height'].squeeze(), targets['height'])
    loss_weather = F.mse_loss(predictions['weather'], targets['weather'])
    loss_state = F.cross_entropy(predictions['state'], targets['state'])
    loss_species = F.cross_entropy(predictions['species'], targets['species'])
    
    # Weighted combination
    total_loss = (
        0.2 * loss_ndvi + 
        0.2 * loss_height + 
        0.3 * loss_weather + 
        0.15 * loss_state + 
        0.15 * loss_species
    )
    
    return total_loss, {
        'ndvi': loss_ndvi.item(),
        'height': loss_height.item(),
        'weather': loss_weather.item(),
        'state': loss_state.item(),
        'species': loss_species.item()
    }

print("✓ Utility functions defined")

✓ Utility functions defined


---
## Part 2: Training Functions

In [30]:
def train_phase1(model, train_loader, val_loader, num_epochs, lr, variation_name):
    """Phase 1: Train to predict tabular features from images."""
    print("\n" + "="*80)
    print(f"PHASE 1: AUXILIARY PRETRAINING - {variation_name}")
    print("="*80 + "\n")
    
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
    history = {
        'train_loss': [], 
        'val_loss': [], 
        'val_ndvi_mae': [], 
        'val_state_acc': [],
        'val_species_acc': []
    }
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = {
                'ndvi': batch['ndvi'].to(device),
                'height': batch['height'].to(device),
                'weather': batch['weather'].to(device),
                'state': batch['state'].to(device),
                'species': batch['species'].to(device)
            }
            
            optimizer.zero_grad()
            predictions = model(images, mode='auxiliary')
            loss, _ = auxiliary_loss(predictions, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        ndvi_errors = []
        state_correct = 0
        state_total = 0
        species_correct = 0
        species_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets = {
                    'ndvi': batch['ndvi'].to(device),
                    'height': batch['height'].to(device),
                    'weather': batch['weather'].to(device),
                    'state': batch['state'].to(device),
                    'species': batch['species'].to(device)
                }
                
                predictions = model(images, mode='auxiliary')
                loss, _ = auxiliary_loss(predictions, targets)
                val_loss += loss.item() * images.size(0)
                
                # NDVI MAE
                ndvi_errors.extend((predictions['ndvi'].squeeze().cpu() - targets['ndvi'].cpu()).abs().tolist())
                
                # State accuracy
                state_pred = predictions['state'].argmax(dim=1)
                state_correct += (state_pred == targets['state']).sum().item()
                state_total += targets['state'].size(0)
                
                # Species accuracy
                species_pred = predictions['species'].argmax(dim=1)
                species_correct += (species_pred == targets['species']).sum().item()
                species_total += targets['species'].size(0)
        
        val_loss /= len(val_loader.dataset)
        ndvi_mae = np.mean(ndvi_errors)
        state_acc = state_correct / state_total
        species_acc = species_correct / species_total
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_ndvi_mae'].append(ndvi_mae)
        history['val_state_acc'].append(state_acc)
        history['val_species_acc'].append(species_acc)
        
        # Print progress
        print(f"Epoch {epoch+1:2d}/{num_epochs}: "
              f"Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, "
              f"NDVI MAE={ndvi_mae:.4f}, State Acc={state_acc:.2%}, Species Acc={species_acc:.2%}")
        
        # Save best checkpoint
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'model4b_{variation_name}_phase1_best.pth')
            print(f"  💾 Saved Phase 1 checkpoint (val_loss={val_loss:.4f})")
    
    print(f"\n✓ Phase 1 complete! Model learned to predict tabular features from images.")
    print(f"  Final State Accuracy: {state_acc:.2%} (model can 'see' location!)")
    print(f"  Final Species Accuracy: {species_acc:.2%}")
    print(f"  Final NDVI MAE: {ndvi_mae:.4f}\n")
    
    return history

print("✓ train_phase1() defined")

✓ train_phase1() defined


In [31]:
def train_phase2(model, train_loader, val_loader, config, target_means, target_stds, variation_name):
    """Phase 2: Fine-tune for biomass prediction with flexible config."""
    print("\n" + "="*80)
    print(f"PHASE 2: BIOMASS FINE-TUNING - {variation_name}")
    print("="*80 + "\n")
    
    num_epochs = config['phase2_epochs']
    lr_head = config['phase2_lr']
    lr_backbone = config['phase2_backbone_lr']
    weight_decay = config['weight_decay']
    use_scheduler = config.get('use_scheduler', False)
    
    print(f"Config: {num_epochs} epochs, head_lr={lr_head:.2e}, backbone_lr={lr_backbone:.2e}, ")
    print(f"        weight_decay={weight_decay:.2e}, scheduler={use_scheduler}")
    
    model = model.to(device)
    
    # Two learning rates: low for pretrained backbone, higher for new head
    optimizer = torch.optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': lr_backbone},
        {'params': model.biomass_head.parameters(), 'lr': lr_head}
    ], weight_decay=weight_decay)
    
    # Optional LR scheduler
    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=config['scheduler_factor'], 
            patience=config['scheduler_patience'], verbose=True
        )
        print(f"✓ LR scheduler enabled (factor={config['scheduler_factor']}, patience={config['scheduler_patience']})")
    
    criterion = nn.MSELoss()
    history = {'train_loss': [], 'val_loss': [], 'val_r2': [], 'per_target_r2': [], 'epoch': []}
    best_r2 = -float('inf')
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            outputs = model(images, mode='biomass')
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets = batch['targets'].to(device)
                targets_original = batch['targets_original']
                
                outputs = model(images, mode='biomass')
                loss = criterion(outputs, targets)
                val_loss += loss.item() * images.size(0)
                
                # Denormalize for R² calculation
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())
        
        val_loss /= len(val_loader.dataset)
        
        # Calculate R²
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        val_r2, per_target_r2 = calculate_competition_r2(all_preds, all_targets)
        
        # Step scheduler if enabled
        if scheduler is not None:
            scheduler.step(val_r2)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_r2'].append(val_r2)
        history['per_target_r2'].append(per_target_r2)
        history['epoch'].append(epoch + 1)
        
        # Print progress
        print(f"Epoch {epoch+1:2d}/{num_epochs}: "
              f"Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val R²={val_r2:+.4f}")
        
        # Save best checkpoint
        if val_r2 > best_r2:
            best_r2 = val_r2
            torch.save(model.state_dict(), f'model4b_{variation_name}_phase2_best.pth')
            print(f"  💾 New best R²={best_r2:+.4f} - checkpoint saved")
    
    print(f"\n✓ Phase 2 complete! Best R²={best_r2:+.4f}")
    print(f"\nPer-target R² scores (at best epoch):")
    best_epoch_idx = np.argmax(history['val_r2'])
    best_per_target = history['per_target_r2'][best_epoch_idx]
    for i, col in enumerate(TARGET_COLS):
        print(f"  {col:15s}: R²={best_per_target[i]:+.4f}")
    print()
    
    return history, best_r2

print("✓ train_phase2() defined (with LR scheduling support)")

✓ train_phase2() defined (with LR scheduling support)


---
## Part 3: Train All EfficientNet Variants

Training 3 EfficientNet variants with different hyperparameters.

In [32]:
# Storage for results
new_results = {}

print("="*80)
print("TRAINING EFFICIENTNET VARIANTS")
print("="*80)
print(f"\nStarting: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if DEBUG_MODE:
    print("\n🐛 DEBUG: 2 epochs/phase (~5 min/variant = ~15 min total)")
else:
    print("\n🚀 FULL: 40 epochs phase2 (~30 min/variant = ~1.5 hours)")

print(f"\nWill train {len(VARIATIONS)} variants")
print()

TRAINING EFFICIENTNET VARIANTS

Starting: 2025-11-06 11:44:39

🚀 FULL: 40 epochs phase2 (~30 min/variant = ~1.5 hours)

Will train 3 variants



In [33]:
# Train each variant
for var_name, config in VARIATIONS.items():
    print("\n\n")
    print("#" * 80)
    print(f"# TRAINING: {var_name}")
    print(f"# {config['description']}")
    print(f"# Expected Kaggle: {config['expected_kaggle']}")
    print("#" * 80)

    # Create datasets
    train_dataset_simple = NormalizedDataset(train_data, target_means, target_stds, augment=True)
    train_dataset_multi = MultimodalDataset(train_data, target_means, target_stds, augment=True)
    val_dataset_simple = NormalizedDataset(val_data, target_means, target_stds, augment=False)
    val_dataset_multi = MultimodalDataset(val_data, target_means, target_stds, augment=False)

    # Create model with BatchNorm support
    model = AuxiliaryPretrainedModel(
        backbone_name=config['backbone'],
        num_outputs=5,
        hidden_dim=config['hidden_dim'],
        dropout=config['dropout'],
        num_states=num_states,
        num_species=num_species,
        use_batch_norm=config.get('use_batch_norm', False)
    )

    print(f"✓ Model: {config['backbone']}, {model.feature_dim} features")
    if config.get('use_batch_norm'):
        print(f"  + BatchNorm enabled in head")

    # Create dataloaders
    train_loader_multi = DataLoader(train_dataset_multi, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_multi = DataLoader(val_dataset_multi, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    train_loader_simple = DataLoader(train_dataset_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_simple = DataLoader(val_dataset_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Phase 1
    history_p1 = train_phase1(
        model=model,
        train_loader=train_loader_multi,
        val_loader=val_loader_multi,
        num_epochs=config['phase1_epochs'],
        lr=config['phase1_lr'],
        variation_name=var_name
    )

    # Phase 2
    history_p2, best_r2 = train_phase2(
        model=model,
        train_loader=train_loader_simple,
        val_loader=val_loader_simple,
        config=config,
        target_means=target_means,
        target_stds=target_stds,
        variation_name=var_name
    )

    # Store results
    new_results[var_name] = {
        'config': config,
        'history_phase1': history_p1,
        'history_phase2': history_p2,
        'best_r2': best_r2,
        'model_checkpoint': f'model4b_{var_name}_phase2_best.pth'
    }

    print(f"\n✅ {var_name} complete! R²={best_r2:+.4f}")
    print(f"   Checkpoint: {new_results[var_name]['model_checkpoint']}")
    print(f"   Expected Kaggle: {config['expected_kaggle']}")




################################################################################
# TRAINING: EfficientNet_HigherLR
# EfficientNet with 3.3x higher LR
# Expected Kaggle: 0.52-0.54
################################################################################
✓ Model: efficientnet_b0, 1280 features

PHASE 1: AUXILIARY PRETRAINING - EfficientNet_HigherLR



Epoch 1/15:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch  1/15: Train Loss=1.1298, Val Loss=1.0625, NDVI MAE=0.4511, State Acc=47.22%, Species Acc=36.11%
  💾 Saved Phase 1 checkpoint (val_loss=1.0625)


Epoch 2/15:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch  2/15: Train Loss=0.7741, Val Loss=0.7503, NDVI MAE=0.4046, State Acc=72.22%, Species Acc=45.83%
  💾 Saved Phase 1 checkpoint (val_loss=0.7503)


...

Epoch 38/40: Train Loss=0.1376, Val Loss=0.3404, Val R²=+0.6361


Epoch 39/40:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 39/40: Train Loss=0.1512, Val Loss=0.3690, Val R²=+0.5935


Epoch 40/40:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 40/40: Train Loss=0.1278, Val Loss=0.3290, Val R²=+0.6496

✓ Phase 2 complete! Best R²=+0.6583

Per-target R² scores (at best epoch):
  Dry_Green_g    : R²=+0.7631
  Dry_Dead_g     : R²=+0.3676
  Dry_Clover_g   : R²=+0.6512
  GDM_g          : R²=+0.7482
  Dry_Total_g    : R²=+0.6610


✅ EfficientNet_WithBatchNorm complete! R²=+0.6583
   Checkpoint: model4b_EfficientNet_WithBatchNorm_phase2_best.pth
   Expected Kaggle: 0.52-0.54


---
## Part 4: Compare Results

Compare the 3 EfficientNet tuning experiments.

In [34]:
print("\n\n")
print("="*80)
print("RESULTS: EFFICIENTNET TUNING")
print("="*80 + "\n")

# Create comparison
comparison_data = []
for var_name, results in new_results.items():
    comparison_data.append({
        'Variant': var_name,
        'Head LR': results['config']['phase2_lr'],
        'Backbone LR': results['config']['phase2_backbone_lr'],
        'BatchNorm': results['config'].get('use_batch_norm', False),
        'Best R²': results['best_r2'],
        'Expected Kaggle': results['config']['expected_kaggle']
    })

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('Best R²', ascending=False)

    print("Results:")
    print(comparison_df.to_string(index=False))

    # Best
    best = comparison_df.iloc[0]
    print(f"\n🏆 BEST: {best['Variant']}")
    print(f"   Val R²: {best['Best R²']:+.4f}")
    print(f"   Expected Kaggle: {best['Expected Kaggle']}")

    # vs Baseline
    baseline_r2 = 0.6852
    baseline_kaggle = 0.51
    improvement = best['Best R²'] - baseline_r2

    print(f"\n📊 vs BASELINE:")
    print(f"   Baseline (ResNet18): Val={baseline_r2:+.4f}, Kaggle={baseline_kaggle:+.2f}")
    print(f"   Best EfficientNet:   Val={best['Best R²']:+.4f}")
    print(f"   Val difference: {improvement:+.4f}")

    if improvement > 0:
        print(f"\n✨ BEATS BASELINE! Should improve Kaggle score!")
    elif improvement > -0.01:
        print(f"\n⚖️  Similar to baseline")
    else:
        print(f"\n📉 Below baseline")
        # Check if it beat original EfficientNet
        orig_eff_r2 = 0.5903
        if best['Best R²'] > orig_eff_r2:
            orig_improvement = best['Best R²'] - orig_eff_r2
            print(f"\n✅ But beats original EfficientNet by {orig_improvement:+.4f}!")
            print(f"   Original EfficientNet: {orig_eff_r2:+.4f}")
            print(f"   This confirms hyperparameter tuning helped!")
else:
    print("No results yet")

print("\n" + "="*80)




RESULTS: EFFICIENTNET TUNING

Results:
                   Variant  Head LR  Backbone LR  BatchNorm  Best R² Expected Kaggle
EfficientNet_WithBatchNorm    0.001      0.00005       True 0.658316       0.52-0.54
   EfficientNet_VeryHighLR    0.002      0.00010      False 0.653211       0.51-0.53
     EfficientNet_HigherLR    0.001      0.00005      False 0.598841       0.52-0.54

🏆 BEST: EfficientNet_WithBatchNorm
   Val R²: +0.6583
   Expected Kaggle: 0.52-0.54

📊 vs BASELINE:
   Baseline (ResNet18): Val=+0.6852, Kaggle=+0.51
   Best EfficientNet:   Val=+0.6583
   Val difference: -0.0269

📉 Below baseline

✅ But beats original EfficientNet by +0.0680!
   Original EfficientNet: +0.5903
   This confirms hyperparameter tuning helped!



---
## Summary

### Original Problem
EfficientNet with ResNet18's hyperparameters: R²=0.5903

### Hypothesis
EfficientNet's 1280 features need higher learning rates than ResNet18's 512

### Results
[See output above]

### Next Steps
If best variant beats baseline (R²>0.6852):
- Create Kaggle submission
- Compare with other experiments

If below baseline but better than original EfficientNet (R²>0.5903):
- Confirms hyperparameter tuning helped
- Consider other architectures (Option 4: K-Fold)